# Agents

The use of LLM's reasoning capacity to perform certain tasks with certain tools at its disposal which the LLM can call/make use of at its own discretion is known as what we call the use of an Agentic Workflow in contrast to automated workflows

In [1]:
import os
from secret_key import openapi_key

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool

In [2]:
os.environ['OPENAI_API_KEY'] = openapi_key

In [38]:
llm = ChatOpenAI(model = "gpt-4", temperature=0, verbose = True)

In [4]:
# create wikipedia tool

wikipedia_tool = WikipediaQueryRun(
    api_wrapper = WikipediaAPIWrapper(
        top_k_results = 2,
        doc_content_chars_max=4000
    )
)

In [5]:
# create calculator tool using @tool decorator

@tool
def calculator(expression:str) -> str:
    """
    Calculate mathematical expressions using Python eval
    Useful for solving math problems. Input should be a valid mathematical
    expression like '2 + 2', '37593 * 67', or '37593**(1/5)'.
    
    Args:
        expression: A mathematical expression to evaluate
        
    Returns:
        The result of the calculation as a string
        
    Examples:
        - "2 + 2" returns "4"
        - "37593 * 67" returns "2518731"
        - "37593**(1/5)" returns "8.222831614237718"
    
    """
    try:
        result = eval(expression.strip())
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {str(e)} "
    

In [39]:
# creating the agent with the new create_agent function

agent = create_agent(
    model = llm,
    tools = [wikipedia_tool, calculator],
    system_prompt = (
        "You are a helpful assistant that can search Wikipedia for information "
        "and perform mathematical calculations. When asked about people, places, "
        "or facts, use the Wikipedia tool. When asked to perform calculations, "
        "use the calculator tool."
    )
)

In [7]:
print("=" * 60)
print("METHOD 1: Using create_agent (LangChain 1.0)")
print("=" * 60)

METHOD 1: Using create_agent (LangChain 1.0)


In [8]:
response = agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "Who is singer Lucky ali's father? How many years have passed since he died"
    }]
})

In [9]:
print(response["messages"][-1].content)
print()

Lucky Ali's father is Mehmood Ali, who was a popular Indian actor known for playing comic roles in Hindi films. He passed away on 23 July 2004, so it has been 18 years since his death.



## Serp API (Google's search API)

In [15]:
from secret_key import serpapi_key
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.tools import Tool

os.environ['SERPAPI_API_KEY'] = serpapi_key

In [17]:
serpapi_tool = Tool(
    name = "google_search",
    description = "Search Google for current information, news, facts, and real-time data. Use this when you need up-to-date information.",
    func=SerpAPIWrapper(
        params={
            "engine": "google",
            "num": 5,  # Number of results (equivalent to top_k_results)
            "gl": "us",  # Country
            "hl": "en"  # Language
        }
    ).run
)

In [18]:
# creating the agent with the new create_agent function

search_agent = create_agent(
    model = llm,
    tools = [serpapi_tool, calculator],
    system_prompt = (
        "You are a helpful assistant that can search google for information "
        "and perform mathematical calculations. When asked about people, places, "
        "or facts, use the serpapi tool. When asked to perform calculations, "
        "use the calculator tool."
    )
)

In [19]:
search_response = search_agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "What was the GDP of Us in 2022 plus 5?"
    }]
})

In [20]:
print(search_response["messages"][-1].content)
print()

The GDP of the US in 2022 was 26.007 trillion US dollars. If you add 5 to this, you get 26,012 trillion US dollars.



# Memory

#### Remembering the context, continuing the chat 

Langchain expression language (LCEL)

Old: SimpleSequentialChain(chains=[chain1, chain2])
New: prompt1 | llm | parser | prompt2 | llm | parser

SequentialChain → RunnableParallel + RunnablePassthrough

ConversationBufferMemory → RunnableWithMessageHistory

In [40]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# 1. Create store for histories
chat_histories = {}

# 2. Define getter function
def get_session_history(session_id: str):
    if session_id not in chat_histories:
        chat_histories[session_id] = InMemoryChatMessageHistory()
    return chat_histories[session_id]

In [41]:
# 3. Wrap agent with memory
agent_with_memory = RunnableWithMessageHistory(
    agent,
    get_session_history,
    input_messages_key="messages",
    history_messages_key="messages"
)

In [42]:
session_id = "user_123"

print("Chat with the agent (type 'exit' to quit)")
print("-" * 50)

while True:
    user_input = input("\nYou: ")
    
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Goodbye!")
        break
    
    response = agent_with_memory.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"session_id": session_id}}
    )
    
    print(f"Agent: {response['messages'][-1].content}")

Chat with the agent (type 'exit' to quit)
--------------------------------------------------
Agent: Sure, I can help with that. Please provide me with the information or calculation you need assistance with.
Agent: Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist best known for developing the theory of relativity. He also made important contributions to quantum theory. His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics for "his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect".

Einstein was born in the German Empire and moved to Switzerland in 1895, where he renounced his German citizenship the following year. He enrolled in the mathematics and physics teaching diploma program at the Swiss federal polytechnic school in Zurich at the age of seventeen, graduating in 1